Item based filtering compute similarity b/t every two items based user purchase behavior. 
we first find the 10 most liked items for user a.
Then we find top 40 most relate items to the top 10 items user a liked.

In [1]:
from MovieLens import MovieLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter
from surprise.model_selection import LeaveOneOut
from RecommenderMetrics import RecommenderMetrics
from EvaluationData import EvaluationData

In [2]:
def LoadMovieLensData():
    ml=MovieLens()
    data=ml.loadMovieLensLatestSmall()
    print('computing movie popularity ranks, later we will use this to measure novelty...')
    rankings = ml.getPopularityRanks()
    return (ml, data, rankings)

In [3]:
ml, data, rankings = LoadMovieLensData()

computing movie popularity ranks, later we will use this to measure novelty...


In [4]:
evalData = EvaluationData(data, rankings)

In [5]:
trainSet= evalData.GetLOOCVTrainSet()

In [6]:
# calculate the similarity matrix. Using surprise package.

sim_options= {'name':'cosine','user_based': False}

model=KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simMatrix= model.compute_similarities()

print(simMatrix[0][:10])


Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
[1.         0.96330706 0.92255278 0.9349689  0.95372125 0.93300536
 0.96062288 0.94569115 0.89626514 0.96964855]


In [7]:
leftOutTestSet= evalData.GetLOOCVTestSet()

In [8]:
# get topN recommendations

topN= defaultdict(list)
k=10

for uiid in range(trainSet.n_users):
    
    # find k items the user liked the most
    userRatings = trainSet.ur[uiid]
    KNeighbors = heapq.nlargest(k, userRatings, key=lambda x: x[1])
    
    # for every item on their top liked, we find items most similiar to that item 
    candidates = defaultdict(float)
    for itemID, rating in KNeighbors:
        SimilarityRow= simMatrix[itemID]
        for innerID, score in enumerate(SimilarityRow):
            candidates[innerID]+= score * (rating/5.0)
            
    # build a dictionary of movies the subject has already watched
    watched={}
    for item, rating in trainSet.ur[uiid]:
        watched[item]=1
        
    # Get top-rated items from similar users that our subject hasn't watched yet: top 40 recommendations

    loop=0
    for item, ratingSum in sorted(candidates.items(),key=itemgetter(1),reverse=True):
        if item not in watched:
            movie_id=trainSet.to_raw_iid(item)
            topN[int(trainSet.to_raw_uid(uiid))].append((int(movie_id), ratingSum))
            loop+=1
            if loop > 40:
                break
    

# evaluating topN using hit rate:


In [9]:
print("HR",RecommenderMetrics.HitRate(topN, leftOutTestSet))

HR 0.004470938897168405
